In [1]:
from pathlib import Path
import os
import json
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [2]:
# from ..tools.preprocess import Preprocessor
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.maneger import DataManager
from datatools.preproc import Preprocessor
# from utterance.feature import Feature
from feature import Feature

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
def json2data(path):
    cols = ["text", "label", "subLabel"]
    df = pd.DataFrame(index=[], columns=cols)
    files = os.listdir(path)
    for cop in files:
        if "." not in cop:
            continue
        with open(path+cop, "r") as f:
            json_data = json.load(f)
            mode = cop.split(".")[0]
            max_ = 300
            for i, data in enumerate( json_data[mode] ) :
                if i > max_:
                    break
                text = data["data"]
                label = data["label"][0]
                # if label == "plain":
                #     break
                subLabel = ""
                df = df.append(pd.DataFrame([text, label, subLabel], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [4]:
label_list = "YN WH please proposal plain".split()
label_dict = dict( zip(label_list, range(len(label_list))) )

def extract_X_y(df):
    X = []
    y = []
    for te, la in zip(df.text, df.label):
        X.append(te)
        # if la == "WH":
        #     y.append(0)
        # else:
        #     y.append(label_dict[la])
        y.append(label_dict[la])
    return X, y
    

In [5]:
corpus_root = "../../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path

'../../corpus/question/'

In [6]:
df = json2data(data_path)
df


,text,label,subLabel
0,メニューを見せていただけますか？,please,
1,おいでいただけますか？,please,
2,マッシュポテトをもらえますか？,please,
3,伝言を預かっていただけますか？,please,
4,ご一緒しませんか？,please,
...,...,...,...
1236,メンバーになりたいかい？,YN,
1237,明日スタートなさいますか？,YN,
1238,トレイがいりますか？,YN,
1239,2階も見たいですか？,YN,


In [7]:
X, y = extract_X_y(df)

In [8]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [11]:
F_path = "../X_y_data/response/"
F_name = "typeClassify_F2.pickle"
featureM = DataManager(F_path)

# if featureM.is_exist(F_name):
#     F = featureM.load_data(F_name)
# else:
#     F = Feature()
#     F.make_features(X_train_str)
#     featureM.save_data(F_name, F)

F = Feature()
F.set_preprocessor(Preprocessor())
F.make_features(X_train_str)
# featureM.save_data(F_name, F)


300


In [12]:
featureM.save_data(F_name, F)

success save : ../X_y_data/response/typeClassify_F2.pickle


In [13]:
print(F.feature_num)
X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

11491


In [14]:
lr = LogisticRegression(solver='sag', max_iter=10000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=10000, solver='sag')

In [15]:
y_pred = lr.predict(X_test)

In [16]:
print(label_dict)
for y_p, x_s in zip(y_pred[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p, x_s))

{'YN': 0, 'WH': 1, 'please': 2, 'proposal': 3, 'plain': 4}
1 : お名前は何とおっしゃいますか？
4 : 先週いっしょにつくったでしょ。
3 : 同じ美容院へ行ったらどう、お父さん?
2 : 本題に入ってくれないか？
2 : 日曜日に、映画にごいっしょしていただけますか？
3 : 青森にいるあなたのお兄さんのヒデキを訪ねるのはどうかしら？
0 : これって本当にあなたなの？
3 : 日本語を習ってみたら？
4 : 申し訳ないけど、知りませんね。
0 : ぼくといっしょに取り組まないか？
0 : そんなことが信じられるだろうか？
1 : エミ叔母ちゃんは何時に来る予定なの？
4 : カイはすごく欲しかったんだ。
0 : もう質問は終わったかい？
2 : それはどんなものだったのか私に説明してもらえますか？
3 : 私といっしょに食卓の用意をしてくれない？
3 : チケットを取りましょうか？
3 : いっしょに散歩に出かけましょうか？
2 : 皆さんに自己紹介をお願いできますか？
0 : そこにはゲームセンターはある？
3 : 「さあ入って、パーティーはちょうど始まるところ。コートを預かりましょうか？」
0 : こちらに座りましょうか？
4 : わたしの家で課題をしましょうよ。
2 : 伝言を預かっていただけますか？
0 : よく眠れた？
4 : どうしてロボコーパスが戦わなくちゃいけないんだ？
0 : 写真をとることはできるかな？
0 : 質問してもいいですか？
4 : ７月25日だよ。
3 : 日本の古い怪談映画の特別プログラムを上映しない？


In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
# print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
# print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
# print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[65  5  8  9  4]
 [ 6 15  0  5  5]
 [ 7  0 46  5  2]
 [ 8  2 10 79  2]
 [ 3  0  1  2 84]]
accuracy =  0.774798927613941


In [18]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()


In [19]:
model_path = "../models/response/"
model_name = "typeClassify_M.pickle"
modelM = DataManager(model_path)
print(model_name)

typeClassify_M.pickle


In [20]:
modelM.save_data(model_name, lr)

success save : ../models/response/typeClassify_M.pickle


In [21]:
# accuracy =  0.7913907284768212
# accuracy =  0.8326446280991735
# accuracy =  0.8367768595041323

In [22]:
# F2_path = "../X_y_data/response/"
# F2_name = "typeClassify_F2.dill"
# featureM2 = DataManager(F2_path, format_="dill")

# F2 = Feature()
# # F2.make_features(X_train_str[:3])
# featureM2.save_data(F2_name, F2)

In [23]:
F2.featurization("そうですね．最近熱いですから")

NameError: name 'F2' is not defined

In [ ]:
# from pyknp import Juman
# from sentence_transformers import SentenceTransformer
# import scipy.spatial
# model_path = "/home/yamada/Downloads/training_bert_japanese"
# Nmodel = SentenceTransformer(model_path, show_progress_bar=False)
# emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

In [ ]:
# X_bert = Nmodel.encode(X_train_str)
# X_bert_test = Nmodel.encode(X_test_str)

In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
est = SVC(kernel="rbf", random_state=2, C=1, gamma=0.01)
clf = OneVsRestClassifier(est)

In [ ]:
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=SVC(C=1, gamma=0.01, random_state=2))

In [ ]:
y_pred_b = clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred_b))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred_b))

confusion matrix = 
 [[62  3 10 13  3]
 [10 15  0  5  1]
 [ 4  0 48  5  3]
 [ 4  2  8 84  3]
 [ 0  0  0  1 89]]
accuracy =  0.7989276139410187


In [ ]:
print(label_dict)
for y_p, x_s in zip(y_pred_b[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p, x_s))

{'YN': 0, 'WH': 1, 'please': 2, 'proposal': 3, 'plain': 4}
1 : お名前は何とおっしゃいますか？
4 : 先週いっしょにつくったでしょ。
3 : 同じ美容院へ行ったらどう、お父さん?
2 : 本題に入ってくれないか？
2 : 日曜日に、映画にごいっしょしていただけますか？
3 : 青森にいるあなたのお兄さんのヒデキを訪ねるのはどうかしら？
0 : これって本当にあなたなの？
3 : 日本語を習ってみたら？
4 : 申し訳ないけど、知りませんね。
3 : ぼくといっしょに取り組まないか？
0 : そんなことが信じられるだろうか？
1 : エミ叔母ちゃんは何時に来る予定なの？
4 : カイはすごく欲しかったんだ。
0 : もう質問は終わったかい？
2 : それはどんなものだったのか私に説明してもらえますか？
3 : 私といっしょに食卓の用意をしてくれない？
3 : チケットを取りましょうか？
3 : いっしょに散歩に出かけましょうか？
2 : 皆さんに自己紹介をお願いできますか？
0 : そこにはゲームセンターはある？
4 : 「さあ入って、パーティーはちょうど始まるところ。コートを預かりましょうか？」
1 : こちらに座りましょうか？
4 : わたしの家で課題をしましょうよ。
2 : 伝言を預かっていただけますか？
3 : よく眠れた？
3 : どうしてロボコーパスが戦わなくちゃいけないんだ？
0 : 写真をとることはできるかな？
0 : 質問してもいいですか？
4 : ７月25日だよ。
3 : 日本の古い怪談映画の特別プログラムを上映しない？


In [ ]:
model2_path = "../models/response/"
model2_name = "typeClassify_SVC.pickle"
model2M = DataManager(model2_path)
print(model2_name)

typeClassify_SVC.pickle


In [ ]:
model2M.save_data(model2_name, clf)

success save : ../models/response/typeClassify_SVC.pickle
